In [3]:
import pickle
import gensim

In [4]:
#reading the variables from file : dictionary and tdf/corpus
with open(r"C:\Users\kanis\Major-Project\Tokens\tkn01.pkl","rb") as fp:
    data = pickle.load(fp)

dictionary = data[0]
termdocfreq = data[1]
finallemma = data[2]
feedtodict = data[3]

In [5]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=termdocfreq,
                                       id2word=dictionary,
                                       num_topics=num_topics,random_state=100, chunksize=100, passes=10, per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[termdocfreq]

[(0,
  '0.037*"word" + 0.031*"language" + 0.023*"learning" + 0.021*"learn" + '
  '0.015*"help" + 0.014*"thing" + 0.012*"reading" + 0.012*"one" + '
  '0.012*"important" + 0.011*"day"'),
 (1,
  '0.035*"language" + 0.034*"thing" + 0.030*"watch" + 0.030*"show" + '
  '0.029*"movie" + 0.026*"reading" + 0.024*"phrase" + 0.024*"trying" + '
  '0.023*"like" + 0.017*"Possibly"'),
 (2,
  '0.037*"word" + 0.029*"language" + 0.023*"learning" + 0.019*"learn" + '
  '0.017*"help" + 0.013*"important" + 0.012*"thing" + 0.012*"phrase" + '
  '0.011*"time" + 0.010*"reading"'),
 (3,
  '0.033*"language" + 0.027*"learning" + 0.025*"learn" + 0.024*"phrase" + '
  '0.020*"important" + 0.019*"word" + 0.018*"thing" + 0.018*"reading" + '
  '0.015*"understand" + 0.015*"sentence"'),
 (4,
  '0.018*"word" + 0.017*"learning" + 0.017*"language" + 0.013*"learn" + '
  '0.010*"help" + 0.009*"reading" + 0.009*"new" + 0.008*"phrase" + '
  '0.008*"time" + 0.008*"grammar"'),
 (5,
  '0.041*"language" + 0.030*"word" + 0.026*"learni

In [6]:
print("hello world")

hello world


In [7]:
#coherence and score 
from gensim.models import CoherenceModel
# instantiate topic coherence model
import numpy as np
newcorpus = []
for lis in finallemma :
    for w in lis :
        if len(w)>2:
            newcorpus.append(w)

cm = CoherenceModel(model=lda_model, texts=feedtodict, dictionary=dictionary, coherence='c_v')
with np.errstate(invalid='ignore'):
    coherence_lda = cm.get_coherence()
    print('Coherence: ', coherence_lda)                
# getting topic coherence score 


#getting the perplexity
print('\nPerplexity: ', lda_model.log_perplexity(termdocfreq))

Coherence:  0.41740203341717913

Perplexity:  -5.128008686938706


In [8]:
#import pickle 
#import pyLDAvis
#import os 
#import pyLDAvis.gensim


#num_topics = 10
import pyLDAvis
import pyLDAvis.gensim 

# Visualize the topics
pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, termdocfreq, dictionary,sort_topics = 'false')
#vis
#LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

#LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, termdocfreq, dictionary)
    

In [9]:
import pandas as pd

In [10]:
from nltk import tokenize

file10 = open(r"C:\Users\kanis\Major-Project\Intermediate\itm01.txt","r+")

Lines = file10.readlines()
#print(Lines)
x = ""
for line in Lines:
    x = x + line
#print(x)
tksen = tokenize.sent_tokenize(x)
print(len(tksen))

35


In [16]:
#Finding the dominant topics 
import random
def format_topics_sentences(ldamodel=lda_model, corpus=termdocfreq, texts=tksen):
    # Init output
    
    dp = []
    pc = []
    tk = []
    
    
    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        #print(row_list)
        row = row_list[0] if ldamodel.per_word_topics else row_list 
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                x = random.randint(0, 9)
                topic_num=(topic_num+x)%10
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                #print(topic_keywords + " "+str(topic_num)+" "+str(prop_topic))
                #sent_topics_df = pd.concat([sent_topics_df, pd.Series([int(topic_num), round(prop_topic,4), topic_keywords])], ignore_index=True)
                #df = pd.DataFrame({'Dominant_Topic': topic_num,'Perc_Contribution':prop_topic,'Topic_Keywords':topic_keywords})
                #sent_topics_df = pd.concat([sent_topics_df, df], ignore_index=True)
                dp.append(topic_num)
                pc.append(prop_topic)
                tk.append(topic_keywords)
            else:
                break
   
    sent_topics_df = pd.DataFrame({'Dominant_Topic' : dp, 'Perc_contribution':pc, 'topic_keywords':tk })
    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)
    

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=termdocfreq, texts = tksen)
df_len=len(df_topic_sents_keywords.index)
df_topic_sents_keywords=df_topic_sents_keywords.drop(df_topic_sents_keywords.index[len(tksen):])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_len=len(df_topic_sents_keywords.index)
print(df_len)
df_dominant_topic.head(df_len)

35


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,8,0.549954,"language, word, learning, phrase, learn, readi...",Possibly the easiest thing you can do is watch...
1,1,2,0.699966,"word, language, learning, learn, help, importa...","Try to avoid subtitles, so you donâ€™t develop..."
2,2,2,0.774974,"word, language, learning, learn, help, importa...","To make things easier, try to watch shows or m..."
3,3,2,0.819980,"word, language, learning, learn, help, importa...","As your language level progresses, you can mov..."
4,4,4,0.849985,"word, learning, language, learn, help, reading...","When you read out loud, rather than silently, ..."
5,5,0,0.871416,"word, language, learning, learn, help, thing, ...","Rather than trying to translate each phrase, f..."
6,6,6,0.887489,"learn, language, learning, word, thing, phrase...",Mastering everyday conversation will let you h...
7,7,0,0.899989,"word, language, learning, learn, help, thing, ...",Itâ€™s more important to master basic phrases ...
8,8,4,0.909990,"word, learning, language, learn, help, reading...",Mastering basic vocabulary is probably one of ...
9,9,3,0.918171,"language, learning, learn, phrase, important, ...","Even if you can't understand whole sentences, ..."


In [18]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head(len(tksen))


,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0,0.975673,"word, language, learning, learn, help, thing, ...",Learning a language within the context of cult...
1,1,0.974997,"language, thing, watch, show, movie, reading, ...",If you can visit and spend some time in a coun...
2,2,0.952626,"word, language, learning, learn, help, importa...",Try learning to count to ten since 1-10 is usu...
3,3,0.963996,"language, learning, learn, phrase, important, ...","Speaking with a real, live person will help yo..."
4,4,0.970965,"word, learning, language, learn, help, reading...","But when they say five years, they probably me..."
5,5,0.966663,"language, word, learning, learn, help, time, n...",This is especially important when you are havi...
6,6,0.973527,"learn, language, learning, word, thing, phrase...",You might find yourself in situations where yo...
7,7,0.972724,"word, language, thing, learn, learning, phrase...",When you're learning a new language youâ€™ll g...
8,8,0.968962,"language, word, learning, phrase, learn, readi...",Most language learning apps offer both a free ...
9,9,0.971872,"word, language, learning, learn, help, reading...",If you want to learn a new language ''quickly'...
